This notebook just reads the raw csv files, joins them, drops duplicate columns, performs formatting, creates aggregate features and saves the output as a new csv file 

Workflow:
data -> imputer -> preprocess -> survival

In [1]:
import numpy as np
import pandas as pd

from load import load_table, to_csv

from utilities import get_cumulative, get_indicator, merge, add_aggregate

In [2]:
path = './Data'

In [3]:
av_tumour = load_table('av_tumour', path)
av_patient = load_table('av_patient', path)
sact_patient = load_table('sact_patient', path)
sact_tumour = load_table('sact_tumour', path)
sact_regimen = load_table('sact_regimen', path)
sact_cycle = load_table('sact_cycle', path)

print(f'AV_TUMOUR rows: {av_tumour.shape[0]}')
print(f'AV_PATIENT rows: {av_patient.shape[0]}')
print(f'SACT_PATIENT rows: {sact_patient.shape[0]}')
print(f'SACT_TUMOUR rows: {sact_tumour.shape[0]}')
print(f'SACT_REGIMEN rows: {sact_regimen.shape[0]}')
print(f'SACT_CYCLE rows: {sact_cycle.shape[0]}')

AV_TUMOUR rows: 2371281
AV_PATIENT rows: 2200626
SACT_PATIENT rows: 366266
SACT_TUMOUR rows: 450072
SACT_REGIMEN rows: 730472
SACT_CYCLE rows: 2442037


In [4]:
sact_cycle.loc[sact_cycle.MERGED_PATIENT_ID == 210009833].shape

(41, 8)

In [5]:
# perform all operations on a copy of av_tumour
df = av_tumour.copy()
df.shape

(2371281, 34)

In [6]:
# Count number of tumour occurrences at and before a given diagnosis date for a particular patient
df = get_cumulative(df, 'PATIENTID', 'DIAGNOSISDATEBEST', 'TUMOUR_COUNT','TUMOURID')
df.shape

lost:  0


(2371281, 35)

In [7]:
# brain ICD10 codes as per gliocova - see Kerlann's email from 3/2/21
# C34 lung
# C50 breast

OPTION = 0
if OPTION == 0:
    df = df.drop(df[(df.SITE_ICD10_O2_3CHAR != 'C70') & 
                    (df.SITE_ICD10_O2_3CHAR != 'C71') & 
                    (df.SITE_ICD10_O2_3CHAR != 'C72') & 
                    (df.SITE_ICD10_O2_3CHAR != 'D32') &
                    (df.SITE_ICD10_O2_3CHAR != 'D33') &
                    (df.SITE_ICD10_O2 != 'D352') &
                    (df.SITE_ICD10_O2 != 'D353') &
                    (df.SITE_ICD10_O2 != 'D354')].index)
elif OPTION == 1:
    df = df.drop(df[(df.SITE_ICD10_O2_3CHAR != 'C34')].index)
elif OPTION == 2:
    df = df.drop(df[(df.SITE_ICD10_O2_3CHAR != 'C50')].index)
df.shape

(41369, 35)

In [8]:
# note the mismatch between SITE_ICD10_O2 and SITE_ICD10_O2_3CHAR 
df.SITE_ICD10_O2_3CHAR.value_counts()
# df[(df.SITE_ICD10_O2 == 'D352')]

C71    20565
D32    11162
D33     4492
D35     4088
C72      689
       ...  
C11        0
C10        0
C09        0
C08        0
D13        0
Name: SITE_ICD10_O2_3CHAR, Length: 115, dtype: int64

In [9]:
df = merge(df, av_patient, on='PATIENTID', suffixes=(None, '_r'))
# list(df.columns) 
df.shape

(41369, 44)

In [10]:
sact_patient['SACT'] = pd.Series([True] * len(sact_patient.index), dtype='boolean')
df = merge(df, sact_patient, how='left', left_on='LINKNUMBER', right_on='LINK_NUMBER', suffixes=(None, '_r'))
df = df.drop(columns='LINK_NUMBER')
# list(df.columns)
df.shape 

(41369, 46)

In [11]:
df = merge(df, sact_tumour, how='left', on='MERGED_PATIENT_ID', suffixes=(None, '_r'))
# list(df.columns) 
df.shape

(47333, 50)

In [12]:
df = merge(df, sact_regimen, how='left', on='MERGED_TUMOUR_ID', suffixes=(None, '_r'))
df.shape

(59263, 60)

In [13]:
# those with multiple regimens
# regimen = df.loc[~df.MERGED_REGIMEN_ID.isnull()]
# multireg = regimen.loc[regimen.PATIENTID.duplicated()]
# multireg

# rows with Regimen
print('rows with Regimen', df.loc[~df.START_DATE_OF_REGIMEN.isnull()].shape[0])

rows with Regimen 23345


In [14]:
# calculate cumulative count of regimens 
df = get_cumulative(df, groupby='MERGED_PATIENT_ID', date='START_DATE_OF_REGIMEN', name='REGIMEN_COUNT', key='MERGED_REGIMEN_ID')
df.shape

lost:  0


(59263, 61)

In [15]:
### Does this even make sense? Perhaps just days since start of diagnosis 
# df['DATE_FIRST_REGIMEN'] = df[['START_DATE_OF_REGIMEN','MERGED_PATIENT_ID']][~df.START_DATE_OF_REGIMEN.isnull()].groupby(['MERGED_PATIENT_ID'])['START_DATE_OF_REGIMEN'].transform('min')
# df.shape


In [16]:
# assuming 02, N mean no; 00, 99 and missing are treated as no as well
df['CLINICAL_TRIAL'] = (df['CLINICAL_TRIAL'] == '01') | (df['CLINICAL_TRIAL'] == '1') | (df['CLINICAL_TRIAL'] == 'Y') | (df['CLINICAL_TRIAL'] == 'y')

df = get_indicator(df,'CLINICAL_TRIAL','PATIENTID','START_DATE_OF_REGIMEN','CLINICAL_TRIAL_INDICATOR','MERGED_REGIMEN_ID') 
df.shape


lost:  0


(59263, 62)

In [17]:
# chemo_radiation from SACT to be kept  
# assuming 2, N mean no; 0 and missing are treated as no as well
df['CHEMO_RADIATION'] = (df['CHEMO_RADIATION'] == '1') | (df['CHEMO_RADIATION'] == 'Y') | (df['CHEMO_RADIATION'] == 'y')

df = get_indicator(df,'CHEMO_RADIATION','PATIENTID','START_DATE_OF_REGIMEN','CHEMO_RADIATION_INDICATOR','MERGED_REGIMEN_ID') 
df.shape
# df['CHEMO_RADIATION_INDICATOR'] = df[['CHEMO_RADIATION_INDICATOR','MERGED_PATIENT_ID', 'MERGED_REGIMEN_ID']][~df.MERGED_REGIMEN_ID.isnull()].groupby(['MERGED_PATIENT_ID'])['CHEMO_RADIATION_INDICATOR'].transform('max')

lost:  0


(59263, 63)

In [18]:
# height and weight
df = add_aggregate(df,what='median',target='HEIGHT_AT_START_OF_REGIMEN',groupby='MERGED_PATIENT_ID',name='NORMALISED_HEIGHT')
df = add_aggregate(df,what='median',target='WEIGHT_AT_START_OF_REGIMEN',groupby='MERGED_PATIENT_ID',name='NORMALISED_WEIGHT')

df.shape
# median_heights = df[(df['SEX'] == 2) & ~(df['HEIGHT_AT_START_OF_REGIMEN'].isnull())]['HEIGHT_AT_START_OF_REGIMEN'].median()

lost:  0
lost:  0


(59263, 65)

In [19]:
df = merge(df, sact_cycle, how='left', on='MERGED_REGIMEN_ID', suffixes=(None, '_r'))
df.shape

(94259, 70)

In [20]:
# get a sensible vital status date that's no earlier than any date in the dataset
df['LATESTDATE'] = df[['DIAGNOSISDATEBEST','DATE_FIRST_SURGERY','VITALSTATUSDATE','DATE_DECISION_TO_TREAT','START_DATE_OF_REGIMEN','START_DATE_OF_CYCLE']].max(axis=1) 

In [21]:
df = df.drop(columns=['CONSULTANT_SPECIALITY_CODE','PRIMARY_DIAGNOSIS','MORPHOLOGY_CLEAN','MERGED_REGIMEN_ID'])
df = df.drop(columns=['HEIGHT_AT_START_OF_REGIMEN','WEIGHT_AT_START_OF_REGIMEN','INTENT_OF_TREATMENT'])
df = df.drop(columns=['MAPPED_REGIMEN','BENCHMARK_GROUP'])
# df = df.drop(columns=['DATE_DECISION_TO_TREAT','START_DATE_OF_REGIMEN','START_DATE_OF_CYCLE'])
# df = df.drop(columns=['CLINICAL_TRIAL','CHEMO_RADIATION'])
# df = df.drop(columns=['MERGED_CYCLE_ID','CYCLE_NUMBER'])
df = df.drop(columns=['OPCS_PROCUREMENT_CODE','PERF_STATUS_START_OF_CYCLE'])
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(94259, 60)
(94259, 60)


In [22]:
if OPTION == 0:
    to_csv(df, f'{path}/bt.csv')
elif OPTION == 1:
    to_csv(df, f'{path}/lung.csv')
elif OPTION == 2:
    to_csv(df, f'{path}/breast.csv')
df.dtypes


TUMOURID                              int64
PATIENTID                             int64
DIAGNOSISDATEBEST            datetime64[ns]
SITE_ICD10_O2                      category
SITE_ICD10_O2_3CHAR                category
MORPH_ICD10_O2                     category
BEHAVIOUR_ICD10_O2                 category
T_BEST                             category
N_BEST                             category
M_BEST                             category
STAGE_BEST                         category
STAGE_BEST_SYSTEM                  category
GRADE                              category
AGE                                 float64
SEX                                category
CREG_CODE                          category
LINKNUMBER                            int64
SCREENINGSTATUSFULL_CODE           category
ER_STATUS                          category
ER_SCORE                           category
PR_STATUS                          category
PR_SCORE                           category
HER2_STATUS                     